## Import Necessary packages

In [1]:
import numpy as np; import pandas as pd
import os, openpyxl; import ipywidgets as widgets
import xlwings as xw ; import pathlib as Path

from ipywidgets import Checkbox, Dropdown, Layout
from IPython.display import Image, display

try:
    from ipyfilechooser import FileChooser
except ModuleNotFoundError:
    print("Installing ipyfilechooser...")
    %pip install ipyfilechooser -q

### Folder & Directory Selection

In [2]:
# Select File paths

rv = FileChooser(
    "..",
    title="<b>Select TEMPLATE Results Viewer:</b>",
    layout=Layout(width="800px"),
    filter_pattern="[!~$]*.xls*",
)
rv._show_dialog()
display(rv)

# Select the folder in which all the test cases are located

# Pick where files are stored locally
results_folder = FileChooser(
    "..",
    title="<b>Select RESOLVE results folder:</b>",
    layout=Layout(width="800px"),
    show_only_dirs=True,
)
results_folder._show_dialog()
display(results_folder)


FileChooser(path='C:\Users\ritvik.jain\PycharmProjects2', filename='', title='<b>Select TEMPLATE Results Viewe…

FileChooser(path='C:\Users\ritvik.jain\PycharmProjects2', filename='', title='<b>Select RESOLVE results folder…

<div class="alert alert-block alert-info">
    ℹ️ <b>How does this script work?</b><br><br>
    RESOLVE raw results, downloaded from AWS s3 consist of multiple CSVs. This script goes through all of the raw Results and load CSVs of interest into a template Result Viewer workbook, recalculates and then spits out a case-specific RV.<br>
    
The user can specify one or multiple cases and then direct the script to the location where these cases are located. <br>
The script will then loop through each of the cases and produce a seperate RV for each of the cases specified. 
    
    
</div>

### Input the names and timestamps of cases you want to generate the RV for

In [3]:
# 'cases_and_timestamps' takes as input a list of tuples, where each tuple represents the case name and timestamp
#  Before running the code blocks below please make sure that the template RV excel file is NOT OPEN!!!!

cases_and_timestamps = [
  ('IEPR Test case', '2024-05-09 05-20-36'),
  ('Core_Forced_All_LLT_2035_AZ_Solar'   ,''),
  ('Core_Forced_LLT_Geo_LDES_2035'   ,'')
 ]

rv_path = rv.selected ; results_path = results_folder.selected

In [8]:
#Following is the list of all csvs that are fed into the scenario tool

sheet_mapping = [
    ('results_summary/asset_summary.csv', 'Asset Summary'),                                 # 1   
    ('results_summary/resource_summary.csv', 'Resource Summary'),                           # 2
    ('results_summary/resource_fuel_burn_summary.csv', 'Resource Fuel Burn'),               # 3
    ('results_summary/annual_load_components_summary.csv', 'Load Components'),              # 4
    ('results_summary/transmission_summary.csv', 'Transmission Summary'),                   # 5
    ('results_summary/zonal_summary.csv', 'Zonal Summary'),                                 # 6
    ('results_summary/policy_summary.csv', 'Policy Summary'),                               # 7
    ('results_summary/prm_policy_resource_summary.csv', 'PRM Resource Summary'),            # 8
    ('results_summary/energy_policy_summary.csv', 'Energy Policy Summary'),                 # 9
    ('results_summary/emissions_policy_tx_path_summary.csv', 'Emissions Policy TxPath'),    # 10
    ('results_summary/emissions_policy_resource_summary.csv', 'Emissions Policy Resource'), # 11
    ('variables/ELCC_MW.csv', 'ELCC_MW'),                                                   # 12
    ('constraints/ELCC_Facet_Constraint_LHS.csv','ELCC_Facet_Constraint_LHS'),              # 13
    ('constraints/Custom_Constraint.csv','Custom Constraints'),                             # 14  
    ('results_summary/temporal_settings_summary.csv', 'Temporal Settings'),                 # 15
    ('temporal_settings/rep_periods.csv','Representative Sample Days'),                     # 16
]

new_rv = 'Resolve RV - {} {}.xlsx'

In [5]:
def generate_rv(cases_and_timestamps= cases_and_timestamps):

    for case, timestamp in cases_and_timestamps:
        if pd.isna(timestamp):
            results_dir = os.path.join(results_path,case)
            #if the RV Already exists then loop out  
            if os.path.exists(os.path.join(results_path,new_rv.format(case))):
                return "Result Viewer Already Exists for {} {}".format(case,timestamp)
        else:
            results_dir = os.path.join(results_path,case,timestamp)
            #if the RV already exists then loop out
            if os.path.exists(os.path.join(results_path,new_rv.format(case,timestamp))):
                return "Result Viewer Already Exists for {} {}".format(case,timestamp)
         
        #once confirmed that the RV does not exist, start loading raw results
        
        wb = xw.Book(rv_path)
        print("Starting RV Creation for {} {}".format(case,timestamp))
        for summary_filename, target_sheet_name in sheet_mapping:
        
            if target_sheet_name not in [sheet.name for sheet in wb.sheets]:
                wb.sheets.add(name=target_sheet_name, after='Raw Summary Files >>')
        
            wb.app.status_bar = f'Loading {summary_filename}'
            df = pd.DataFrame()
            df = pd.concat([df, pd.read_csv(os.path.join(results_dir, summary_filename))], ignore_index=True)
            print(f"Done with {target_sheet_name}") 
            wb.sheets[target_sheet_name].api.AutoFilterMode = False
            wb.sheets[target_sheet_name].clear()
            wb.sheets[target_sheet_name].range('A1').options(index=False).value = df
        
        print(f"Done reading raw files for {case} {timestamp} ")
        wb.sheets['Dashboard'].range('active_case').value = f"{case}_{timestamp}"
        wb.app.status_bar = f'Finished loading {case} {timestamp}'

        wb.save(path = os.path.join(results_path,new_rv.format(case,timestamp)))
        
    return wb.close()

In [6]:
def generate_single_rv(cases_and_timestamps):
    generate_rv(cases_and_timestamps)
    
    return "All RV's produced"
    
def generate_multiple_rvs(cases_and_timestamps):
    for _ in cases_and_timestamps:
        generate_rv([tuple(_)])
    
    return "All RV's produced"

In [7]:
generate_multiple_rvs(cases_and_timestamps)

Starting RV Creation for Core_Forced_All_LLT_2035_AZ_Solar 
Done with Asset Summary
Done with Resource Summary
Done with Resource Fuel Burn
Done with Load Components
Done with Transmission Summary
Done with Zonal Summary
Done with Policy Summary
Done with PRM Resource Summary
Done with Energy Policy Summary
Done with Emissions Policy TxPath
Done with Emissions Policy Resource
Done with ELCC_MW
Done with ELCC_Facet_Constraint_LHS
Done with Custom Constraints
Done with Temporal Settings
Done with Representative Sample Days
Done reading raw files for Core_Forced_All_LLT_2035_AZ_Solar  
Starting RV Creation for Core_Forced_LLT_Geo_LDES_2035 
Done with Asset Summary
Done with Resource Summary
Done with Resource Fuel Burn
Done with Load Components
Done with Transmission Summary
Done with Zonal Summary
Done with Policy Summary
Done with PRM Resource Summary
Done with Energy Policy Summary
Done with Emissions Policy TxPath
Done with Emissions Policy Resource
Done with ELCC_MW
Done with ELCC_Fa

"All RV's produced"

In [17]:
# All the case specific RV's should now be produced in the results folder where the cases are saved

## Case Comparison (Optional)

In [18]:
# Select the comparison Workbook template 
case_comp = FileChooser(
    "..",
    title="<b>Select TEMPLATE Case Comparison Workbook:</b>",
    layout=Layout(width="800px"),
    filter_pattern="[!~$]*.xls*",
)
case_comp._show_dialog()
display(case_comp)

FileChooser(path='C:\Users\ritvik.jain\PycharmProjects2', filename='', title='<b>Select TEMPLATE Case Comparis…

<div class="alert alert-block alert-info">
    ℹ️ <b>How does case comparison work?</b><br><br>
    For RESOLVE batch runs it is often important to compare and contrast different cases for QA/QC, sensitivities testing etc. <br>
    
After the creation of the RV, the user can define what cases they want to compare and load the "Summaries" from each of those cases on to a Case Comparison Excel Workbook <br> <br>
Once that is done the user can easily toggle comparisons between different cases from the excel workbook.     
    
</div>


### Considering that you have used the above script to generate RV's and are continuing the same process to do case comparisons

In [19]:
results_path = results_folder.selected ; 
rvs = [file for file in os.listdir(results_path) if file.endswith('.xlsx') and "Resolve RV" in file]

In [22]:
def create_case_comp_workbook():
    
    comp_wb = xw.Book(case_comp.selected)

    for rv in rvs:

        rv_temp = xw.Book(os.path.join(results_path,rv))
        case_name = rv_temp.sheets['Dashboard'].range('active_case').value
        if len(case_name) >31:
            print("Case Name is too long to be a Sheet name. Name will be shortened and saved")
            case_name = case_name[:30]
        print("Loading Summary tab for {}".format(case_name))

        target_sheet_name = case_name
        if target_sheet_name not in [sheet.name for sheet in comp_wb.sheets]:
            rv_temp.sheets['Summary'].api.Copy(After=comp_wb.sheets['Data >>'].api)
            comp_wb.sheets['Summary'].name = target_sheet_name
            case_data = comp_wb.sheets[target_sheet_name].range('A1:AG465').value
            comp_wb.sheets[target_sheet_name].range('A1').value = case_data
            print("Sheet created for {}".format(case_name))

        else:
            print("Sheet already exists for {}".format(target_sheet_name))

        rv_temp.close()

        print('Now Saving')
        comp_wb.save()

    print("Summary of all cases loaded on to the Case Comparison Worksheet!")
    return comp_wb

In [23]:
create_case_comp_workbook()

Case Name is too long to be a Sheet name. Name will be shortened and saved
Loading Summary tab for Core_Forced_All_LLT_2035_AZ_So
Sheet created for Core_Forced_All_LLT_2035_AZ_So
Now Saving
Loading Summary tab for Core_Forced_LLT_Geo_LDES_2035_
Sheet created for Core_Forced_LLT_Geo_LDES_2035_
Now Saving
Case Name is too long to be a Sheet name. Name will be shortened and saved
Loading Summary tab for IEPR Test case_2024-05-09 05-2
Sheet created for IEPR Test case_2024-05-09 05-2
Now Saving
Summary of all cases loaded on to the Case Comparison Worksheet!


<Book [Comparison Workbook Template.xlsx]>